# python web 框架入门——FASTAPI

## web框架概览
| 特性            | FastAPI               | Flask                 | Django                   |
|-----------------|-----------------------|-----------------------|--------------------------|
| **发布**        | 2018                  | 2010                  | 2005                     |
| **设计理念**    | 高性能API框架         | 小而精的微框架       | 大而全的全包式框架       |
| **主要特性**    | 异步支持、自动数据验证、自动生成文档 | 轻量级、扩展性强    | ORM、认证系统、内置管理后台 |
| **性能**        | 极高（与Go比肩）      | 中等                  | 中等                     |
| **类型提示**    | 原生支持              | 通过扩展支持         | 通过扩展支持             |
| **异步编程**    | 原生支持              | 通过扩展部分支持     | Django 3.1+部分支持      |
| **自动文档**    | 原生支持              | 通过扩展实现         | 通过扩展实现             |
| **适用场景**    | 高性能API、机器学习接口、实时数据处理 | 小到中型项目、微服务 | 大型应用、CMS、电商平台 |


## FastAPI的相关依赖
### 依赖1：Python 3.6 + （负责编码开发）

FastAPI充分利用了Python 3.6+版本的现代特性，尤其是类型提示功能，从而显著提升了开发效率。

这种类型提示不仅使代码更加清晰易读，还有助于静态分析工具准确检测潜在错误，进一步优化整体性能。




### 依赖2：Starlette（负责Web部分）

FastAPI的卓越性能是建立在Starlette框架的坚实基础之上的。

Starlette，作为一个轻量级且专为速度和性能打造的ASGI框架/工具集，为FastAPI提供了强大的异步请求处理能力。

FastAPI自设计之初就全面支持异步编程，优化了底层架构和功能实现，使其在处理大量并发请求时具备极高的效率。

异步I/O操作避免线程阻塞，智能释放线程处理其他任务，实现服务器资源最大化利用，从而在高并发场景下表现卓越，提升服务器响应速度。

这种能力不仅确保了FastAPI在处理请求时的流畅性和高效性，更使得它在众多API框架中脱颖而出，成为了高性能特性的典范。


### 依赖3：Pydantic（负责数据验证部分）

FastAPI巧妙地整合了Pydantic库，利用其强大的数据验证和管理设置功能。

Pydantic基于Python的类型提示，为数据验证提供了快速且可靠的支持。

得益于Pydantic的高效数据处理能力，FastAPI能够迅速解析请求并验证数据的准确性，从而在确保性能的同时，极大地提升了开发效率和用户体验。




### 依赖4：第三方ORM（数据持久层）模块

FastAPI没有内置ORM（对象关系映射）是因为它本身是一个轻量级的框架，专注于提供REST接口的优化和自动生成openapi文档等功能。

可以借助SQLAlchemy模块完成ORM（高版本支持异步）



可以借助Tortoise-orm模块完成ORM（原生支持异步）



## FastAPI的字段工厂

### Query
Query 是用于定义查询字符串参数的工厂类。它的主要参数有：

default: 参数的默认值。如果请求中没有提供该参数，将使用此默认值。

description: 参数的描述，通常用于文档和API接口描述。

max_length: 参数的最大长度。如果提供的参数超过这个长度，将引发一个验证错误。

min_length: 参数的最小长度。如果提供的参数短于这个长度，将引发一个验证错误。

regex: 用于验证参数的正则表达式。

alias: 参数的别名，允许在请求中使用不同的名称来传递此参数。

ge: 大于或等于某个值。

gt: 大于某个值。

le: 小于或等于某个值。

lt: 小于某个值。

multiple: 允许参数在查询字符串中出现多次，返回一个列表。

dependency: 用于创建更复杂的依赖关系，通常与 Depends 结合使用。

In [ ]:
from fastapi import FastAPI, Query
from typing import List

app = FastAPI()

# 使用Query工厂类限制查询参数字段
@app.get("/items/")
async def read_items(
        q: str = Query(None, description="用于筛选项目的搜索查询", max_length=50, min_length=3, alias="搜索查询"),
        size: int = Query(10, description="返回的数据条数", ge=1, le=100),
        skip: int = Query(0, description="跳过的数据条数", ge=0),
        multiple_param: List[str] = Query(None, description="物品清单", multiple=True)
):
    """
    模拟根据查询参数从数据库中读取项目
    """
    results = {
        "query": q,
        "size": size,
        "skip": skip,
        "multiple_param": multiple_param
    }
    return results

# 运行FastAPI应用
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8080)


### Field
Field 通常用于 Pydantic 模型中定义字段，但在 FastAPI 中也可以用于查询参数。它的参数与 Query类似，但主要用于模型字段的描述和验证。

default: 字段的默认值。

description: 字段的描述。

max_length: 字段的最大长度。

min_length: 字段的最小长度。

pattern: 用于验证字段的正则表达式。

alias: 字段的别名。

title: 字段的标题，通常用于文档和模型描述。

ge: 大于或等于某个值。

gt: 大于某个值。

le: 小于或等于某个值。

lt: 小于某个值

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel, Field

app = FastAPI()

class Item(BaseModel):
    name: str = Field(default=..., title="物品名称", max_length=100, min_length=3, description="必须仅包含字母", pattern=r'^[a-zA-Z]+$')
    quantity: int = Field(default=..., ge=0, description="物品数量，长度必须为非负数")
    price: float = Field(default=..., gt=0, le=1000, description="物品价格，必须大于0且小于等于1000")
    description: str = Field(default=None, max_length=500, min_length=10, description="物品的详细描述")

@app.post("/items/")
async def create_item(item: Item):
    """
    模拟根据提供的数据创建新物品保存到数据库
    """
    return {"message": "Item created successfully", "item": item.dict()}

# 运行FastAPI应用
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8080)


### Path
Path 用于定义路径参数。它的参数包括：

default: 参数的默认值。

description: 参数的描述。

max_length: 参数的最大长度。

min_length: 参数的最小长度。

pattern: 用于验证参数的正则表达式。

alias: 参数的别名。

ge: 大于或等于某个值。

gt: 大于某个值。

le: 小于或等于某个值。

lt: 小于某个值。

In [ ]:
from fastapi import FastAPI, Path

app = FastAPI()

# 使用Path工厂类限制路径参数
@app.get("/items/{item_id}")
async def read_item(
        item_id: int = Path(default=..., description="限制路径参数", max_value=100, min_value=1, ge=1, gt=0, alias="item-id"),
):
    """
    模拟根据物品ID读取项目
    """
    results = {
        "item_id": item_id,
    }
    return results

# 运行FastAPI应用
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8080)


### Form
Form 是一个工厂类，用于处理表单数据，通常用于处理 POST 请求中的表单数据。

default：默认值。如果表单中没有包含该字段，将使用此默认值。

description：参数的描述，用于API文档生成。

alias：参数的别名。允许你指定一个不同于字段名的名称来接收表单数据。

max_length：字符串字段的最大长度。

min_length：字符串字段的最小长度。

gt：字段值必须大于指定的值。

ge：字段值必须大于等于指定的值。

lt：字段值必须小于指定的值。

le：字段值必须小于等于指定的值。

pattrn：字段值必须匹配指定的正则表达式。

In [ ]:
from fastapi import FastAPI, Form

app = FastAPI()

# 使用Form工厂类处理表单数据
@app.post("/items/")
async def create_item(
        username: str = Form(..., description="用户名", max_length=100),
        password: str = Form(..., description="密码", min_length=8, alias="pwd"),
):
    """
    模拟根据提供的数据创建新用户保存到数据库
    """
    return {"message": "User created successfully"}

# 运行FastAPI应用
if __name__ == "__main__":


## Fastapi的请求体

### 嵌套模型


In [ ]:
from fastapi import APIRouter
from pydantic import BaseModel, Field, field_validator
from datetime import date
from typing import List, Union, Optional

app03 = APIRouter()

class Addr(BaseModel):
    province: str
    city: str

class User(BaseModel):
    name: str
    age: int = Field(default=18, ge=18, le=60)
    friends: List[int]  # 列表中的元素要求必须全是int类型
    birthday: Optional[date] = None
    description: Union[str, None] = None
    addr: Addr

    @field_validator("name")
    def name_must_alpha(cls, value):
        assert str(value).isalpha(), "name must be alpha"
        return value

class Data(BaseModel):
    data: List[User]

@app03.post("/user")
async def data(user: User):
    print(type(user), user)  # <class 'apps.app03.User'> name='string' age=0 birthday=datetime.date(2024, 3, 15) friends=[0]
    print(user.name)    # zhangsan
    print(user.age)     # 18
    print(user.dict())  # {'name': 'zhangsan', 'age': 18, 'birthday': datetime.date(2024, 3, 15), 'friends': [1, 2, 3]}
    print(user.json())  # {"name":"zhangsan","age":18,"birthday":"2024-03-15","friends":[1, 2, 3]}
    return user

@app03.post("/data")
async def data(data: Data):
    return data


这种情况下，FastAPI将期望得到这样的请求体：

In [ ]:
{
  "name": "string",
  "age": 18,
  "friends": [0],
  "birthday": "2024-03-17",
  "description": "string",
  "addr": {
    "province": "string",
    "city": "string"
  }
}


### Form 表单

In [ ]:
from fastapi import FastAPI, Form

app = FastAPI()

@app.post("/regin")
async def regin(username: str = Form(...), password: str = Form(...)):
    print(f"username: {username}, password: {password}")  # username: admin, password: 123123
    return {"username": username}

# 请求信息
# curl -X 'POST' \
#   'http://127.0.0.1:8080/regin' \
#   -H 'accept: application/json' \
#   -H 'Content-Type: application/x-www-form-urlencoded' \
#   -d 'username=admin&password=123123'


### 文件上传

In [ ]:
from fastapi import FastAPI, File, UploadFile
from typing import List

app = FastAPI()

# 适合小文件上传
@app.post("/files/")
async def create_file(file: bytes = File()):
    print("file:", file)
    return {"file size": len(file)}

@app.post("/multiFiles/")
async def create_files(files: List[bytes] = File()):
    return {"file sizes": [len(file) for file in files]}

# 适合大文件上传
@app.post("/uploadFile/")
async def create_upload_file(file: UploadFile):
    with open(f"{file.filename}", 'wb') as f:
        for chunk in iter(lambda: file.file.read(1024), b''):
            f.write(chunk)
    return {"filename": file.filename}

@app.post("/multiUploadFiles/")
async def create_upload_files(files: List[UploadFile]):
    return {"filenames": [file.filename for file in files]}


### request对象

In [ ]:
from fastapi import APIRouter, Request

app06 = APIRouter()

@app06.get("/items")
async def items(request: Request):
    return {
        "请求URL": request.url,
        "请求IP": request.client.host,
        "请求端口": request.client.port,
        "请求协议": request.scope.get("type"),
        "请求方式": request.method,
        "请求路由": str(request.scope.get("route")),
        "协议版本": request.scope.get("http_version"),
        "请求宿主": request.headers.get("user-agent"),
        "cookies": request.cookies,
        "json数据": await request.json()
    }


## Fastapi的响应模型

### 文件下载响应

In [ ]:
from fastapi import FastAPI
from fastapi.responses import StreamingResponse

app = FastAPI()

# 利用StreamingResponse按需发送文件内容
def file_chunk_iter(file_path, chunk_size=1024):
    with open(file_path, "rb") as file:
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            yield chunk

@app.get("/stream-file/")
def stream_file(file_path: str):
    return StreamingResponse(file_chunk_iter(file_path), media_type="application/octet-stream")

# 运行FastAPI应用
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8080)


### 自定义响应模型

In [ ]:
from fastapi import APIRouter
from pydantic import BaseModel, EmailStr
from typing import Union

app07 = APIRouter()

class UserIn(BaseModel):
    username: str
    password: str
    email: EmailStr
    full_name: Union[str, None]

class UserOut(BaseModel):
    username: str
    email: EmailStr

@app07.post("/createUser1")
async def create_user(user: UserIn):
    # 业务逻辑处理...
    # 存入数据库...
    return user

@app07.post("/createUser2", response_model=UserOut)
async def create_user_response(user: UserIn):
    # 业务逻辑处理...
    # 存入数据库...
    return user


## 依赖注入depends

### 概念和作用：

Depends 允许你定义一个函数，该函数返回一个值（通常是某个对象或数据），这个值可以在后续的路由操作中被使用。更重要的是，Depends 还允许你指定依赖项是否必须存在，以及如何处理依赖项的解析失败。

在 FastAPI 中，你可以使用 Depends 来实现以下功能：

### 数据验证和转换：

可以定义一个 Depends 函数，用于验证和转换请求中的数据，确保它们符合预期的格式和类型。

### 权限和身份验证：

可以使用 Depends 来实现权限控制和身份验证逻辑。

例如，可以定义一个 Depends 函数来检查用户的身份和权限，如果验证失败，则返回相应的错误响应。

### 数据库会话管理：

在处理数据库相关的请求时，可能需要确保每个请求都使用相同的数据库会话。

通过 Depends，可以定义一个函数来创建和管理数据库会话，并在每个路由操作中注入这个会话。

### 请求头、查询参数等的处理：

可以使用 Depends 来处理请求头、查询参数等，提取需要的信息并在路由操作中使用。

In [ ]:
from fastapi import FastAPI, Depends, HTTPException, status
from pydantic import BaseModel

app = FastAPI()

# 定义一个 Pydantic 模型用于数据验证
class User(BaseModel):
    username: str
    password: str

# 定义依赖函数用于验证用户身份
def get_current_user(user: User) -> str:
    if user.username != "admin" or user.password != "password":
        raise HTTPException(status_code=401, detail="Incorrect username or password")
    return user.username

# 定义一个路由操作，使用 Depends 进行依赖注入
@app.post("/items/")
async def create_item(current_user: str = Depends(get_current_user)):
    return {"user": current_user}

# 运行应用
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)


### dependence依赖项
在 FastAPI 中，直接在视图函数参数中声明依赖项和使用dependencies参数在装饰器中声明依赖项之间有一些区别：

视图函数参数中声明依赖项

当依赖项可以直接从请求中提取数据（如路径参数、查询参数、请求头等）时，这种方式非常有用。FastAPI 会自动处理这些依赖项，并将解析后的值作为参数传递给视图函数。这种方式简洁明了，适用于那些每个请求都需要执行且没有特殊依赖顺序的依赖项。

装饰器中的dependencies参数声明依赖项

使用dependencies参数可以让你更灵活地控制依赖项的执行顺序和条件。这对于那些需要根据其他依赖项的结果来决定是否执行某个依赖项，或者需要按照特定顺序执行多个依赖项的场景非常有用。此外，dependencies参数还支持异步依赖项，这是直接在参数中声明依赖项所不支持的。

In [ ]:
from fastapi import FastAPI, Header, Depends, HTTPException
from pydantic import BaseModel, Field

app = FastAPI()

def get_current_user_token(token: str = Header(None)):
    if token == "admin":
        return {"username": "admin", "password": "admin", "token": "admin"}
    else:
        raise HTTPException(status_code=401, detail="只准输入admin哦，这是我们的秘密！")

def get_current_user_level(level: str = Header(None)):
    if level == "admin":
        return {"level": "admin"}
    elif level == "guest":
        return {"level": "guest"}
    else:
        raise HTTPException(status_code=401, detail="出门在外身份都是自己给的！")

@app.get("/users/me1", dependencies=[Depends(get_current_user_token), Depends(get_current_user_level)])
async def read_user_me1():
    # dependencies内两个依赖项执行成功后进入视图函数
    return {"message": "很好，很高兴见到你！"}

@app.get("/users/me2")
async def read_user_me2(
    user_info: dict = Depends(get_current_user_token),
    user_level: dict = Depends(get_current_user_level)
):
    return_info = {"message": "很好，很高兴见到你！"}
    return_info.update(user_info)
    return_info.update(user_level)
    return return_info

# 运行FastAPI应用
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8080)


## 后台任务
FastAPI 的后台任务功能基于 Python 的异步编程模型，巧妙地实现了在处理 HTTP 请求的同时，异步处理那些不需要实时响应但需要在 API 请求完成后执行的任务。

这些任务涵盖了广泛的场景，如发送电子邮件、处理图像、执行复杂的数据分析等。

即使后台任务需要消耗一定的时间和计算资源，也不会对 API 的实时性能和用户体验产生任何影响。

In [ ]:
import time
from fastapi import FastAPI, BackgroundTasks, Depends
from typing import Optional

app = FastAPI()

# 创建一个后台任务函数
async def bg_task(framework: str):
    with open("README.md", mode="a", encoding="utf-8") as f:
        for i in range(1, 10):
            time.sleep(1)
            f.write(f"### 标题{i}\n")
            f.write(f"> 标题{i} 的内容：我是后台任务，{framework}\n")

# 标准后台任务示例
@app.post("/background_tasks")
async def run_bg_task(framework: str, background_tasks: BackgroundTasks):
    """
    :param framework: 被调用的后台任务函数的参数
    :param background_tasks: FastAPI.BackgroundTasks
    :return:
    """
    # 添加到后台任务队列
    background_tasks.add_task(bg_task, framework)
    return {"message": "任务已在后台执行"}

def continue_write_readme(background_tasks: BackgroundTasks, q: Optional[str] = None):
    if q:
        background_tasks.add_task(bg_task, f"\n> 我是 Depends 依赖函数，我要写入：{q}")
    return q

# 依赖后台任务示例
@app.post("/dependency/background_tasks")
async def dependency_run_bg_task(q: str = Depends(continue_write_readme)):
    if q:
        return {"message": "README.md更新成功"}
    return {"message": "没有任务"}

# 启动应用程序
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)


## 异常处理
FastAPI 提供了一个方便且灵活的方式来处理异常错误；

当路由函数或依赖项抛出异常时，FastAPI 会捕获这些异常，并将其转换为适当的 HTTP 响应；

FastAPI 还允许你自定义错误处理，以返回自定义的错误响应；

### 内置异常处理类
HTTPException：基础的 HTTP 异常类，可以用它来抛出自定义的 HTTP 错误。

HTTPException：的两个常用属性：

status_code：可以设置或读取 HTTP 状态码

detail：用于提供关于错误的详细信息

### 自定义异常处理
可以通过继承HTTPException类创建自定义异常，并在路由或依赖中抛出它们

In [ ]:
from fastapi import FastAPI, HTTPException, status
from fastapi.responses import JSONResponse

app = FastAPI()

# 自定义异常类
class ItemNotFound(HTTPException):
    def __init__(self, item_id: str):
        super().__init__(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Item {item_id} not found",
        )

# 错误处理器
@app.exception_handler(ItemNotFound)
async def item_not_found_handler(request, exc: ItemNotFound) -> JSONResponse:
    return JSONResponse(
        status_code=exc.status_code,
        content={"message": exc.detail}
    )

# 路由函数使用内置异常
@app.get("/items/{item_id}")
async def read_item(item_id: int):
    items = {1: "foo", 2: "bar"}
    if item_id not in items:
        raise HTTPException(status_code=404, detail="Item not found")
    return {"item": items[item_id]}

# 路由函数使用自定义异常
@app.get("/custom-error/{item_id}")
async def read_custom_error(item_id: int):
    if item_id != 1:
        raise ItemNotFound(item_id)
    return {"item": "custom error item"}

# 启动应用程序
if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="127.0.0.1", port=8000)


## 中间件

“中间件”是一个函数，它在每个请求被特定的路径操作处理之前,以及在每个响应返回之前介入；

中间件的作用：中间件可以检查请求的内容，如头部信息、查询参数、路径参数等，并据此执行相应的逻辑。 同样，它也可以在响应被发送回客户端之前修改或增强响应。

中间件的流程：中间件首先接收请求，执行一些操作，然后将请求传递给应用程序的其他部分；当应用程序生成响应后，中间件再次介入，可能执行一些额外的操作，然后返回响应。

中间件链：在许多Web框架中，中间件可以形成一个链式结构。一个请求会依次经过多个中间件，每个中间件都会按照上述流程处理请求和响应。这允许开发者在应用程序的不同层次上应用不同的逻辑和策略。

中间件的应用场景：中间件可以应用于许多场景，如身份验证、日志记录、异常处理、缓存、CORS策略等。通过中间件，开发者可以以一种模块化和可重用的方式实现这些功能，而不需要在每个路由或控制器中重复相同的代码。

中间件和路由的关系：在FastAPI或Starlette这样的框架中，中间件和路由是相辅相成的。路由定义了应用程序的端点（即特定的URL路径），而中间件则负责在这些端点之前和之后执行额外的逻辑。这使得开发者可以灵活地组合不同的中间件和路由，以满足复杂的应用程序需求。

在视图函数的顶部使用@app.middleware("http")创建中间件
中间件参数说明：

request：当前HTTP请求对象；

call_next：一个可调用的对象，它代表了中间件链中的下一个中间件或最终的路由操作；

如果该中间件后面还有中间件，那么call_next就是下一个中间件；
如果没有，那么call_next就是对应的视图函数；

In [ ]:
import time
import uvicorn
from fastapi import FastAPI, Request

# 主应用路由（FastAPI实例）
app = FastAPI()

@app.middleware("http")
async def m2(request: Request, call_next):
    # 请求代码卡
    print("m2 request")
    response = await call_next(request)
    # 响应代码卡
    print("m2 response")
    return response

@app.middleware("http")
async def m1(request: Request, call_next):
    # 请求代码卡
    print("m1 request")

    # 限制IP
    # if request.client.host in ["127.0.0.1", ]:  # 黑名单
    #     return Response(content="黑名单")

    # 限制路由
    # if request.url.path in ["/user"]:
    #     return Response(content="黑名单")

    # 记录请求时间
    start_time = time.time()
    response = await call_next(request)

    # 响应代码块
    print("m1 response")
    # 记录响应时间
    end_time = time.time()
    # 计算耗时
    take_time = end_time - start_time
    response.headers["X-start_time"] = str(start_time)
    response.headers["X-end_time"] = str(end_time)
    response.headers["X-take_time"] = str(take_time)

    return response

@app.get("/user")
def get_user():
    print("get_user函数执行了...")
    return {
        "user": "current user"
    }

@app.get("/item/{item_id}")
def get_item(item_id: int):
    print("get_item函数执行了...")
    return {
        "item": f"current item, {item_id}"
    }

if __name__ == '__main__':
    uvicorn.run("main:app", port=8080, reload=True)


### 典型的应用场景

In [ ]:
import uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

# 主应用路由（FastAPI实例）
app = FastAPI()

# 使用FastAPI已经封装完善的跨域解决
origins = [
    "http://localhost:63342"  # 允许的跨域源
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,  # 允许跨域的源列表，例如 ["http://localhost:8080"]
    allow_credentials=True,  # 是否支持 cookies
    allow_methods=["GET"],  # 允许的跨域请求 HTTP 方法列表
    allow_headers=["*"],  # 允许的跨域请求 HTTP 请求头列表
)

@app.get("/user")
def get_user():
    print("get_user函数执行了...")
    return {
        "user": "current user"
    }

if __name__ == '__main__':
    uvicorn.run("main:app", port=8080, reload=True)


## Pydantic

Pydantic 是一个数据验证和设置管理库，它的核心概念是数据模型（Model）和验证（Validation）；

数据模型是通过 Python 类型提示来定义的类，这些类在实例化时会自动使用Python类型注解进行数据验证；确保传入的数据类型符合预期，并且可以自动转换数据类型（比如从字符串到整数）。

### 核心功能
数据验证：Pydantic 可以确保输入数据符合预定义的模型要求。例如，如果定义了一个模型，其中某个字段是整数类型，并且必须大于零，Pydantic 会在接收到这个字段的数据时进行验证，确保数据满足这些条件。如果数据不满足要求，Pydantic 会引发一个错误。

数据转换：Pydantic 可以自动将输入数据转换为模型所需的数据类型。例如，如果模型中的一个字段被定义为整数类型，但用户输入了一个字符串形式的数字，Pydantic 会自动将这个字符串转换为整数。

数据序列化：Pydantic 可以将模型实例转换为字典、JSON 或其他格式的数据，这对于 API 开发、数据存储和传输等场景非常有用。

依赖注入：Pydantic 支持依赖注入，这意味着你可以在模型定义中指定某些字段的值依赖于其他字段的值。这在处理复杂的数据模型时非常有用。

类型提示：由于 Pydantic 基于 Python 的类型提示系统，因此它可以与 Python 的类型检查工具（如 mypy、typing）一起使用，以提高代码的可读性和可维护性。

错误处理：提供清晰、结构化的错误消息，帮助开发者快速定位和修复问题；

设置管理：从配置文件、环境变量等来源加载和验证设置；

### BaseModel核心类
BaseModel 是 Pydantic 库中的一个核心类，用于定义数据模型。

通过继承 BaseModel，可以创建具有数据验证、类型检查、序列化和反序列化等功能的模型。

名称	描述
dict()	返回模型的字典表示
json()	返回模型的JSON字符串表示
copy()	返回模型的浅拷贝
parse_obj()	类方法，用于将已有的字典转换为模型实例
parse_raw()	类方法，用于将JSON或其他格式的字符串转换为模型实例
parse_file()	类方法，用于将JSON或其他格式的文件内容转换为模型实例
from_orm()	类方法，用于从兼容ORM对象创建模型实例。这允许Pydantic模型与ORM模型（如SQLAlchemy）无缝协作，通过将ORM模型实例传递给此方法来创建Pydantic模型实例。
schema()	类方法，返回模型的JSON Schema表示
schema_json()	类方法，返回模型的JSON Schema的JSON字符串表示
construct()	类方法，用于创建模型实例，不执行验证
fields	一个字典，包含模型的所有字段信息。这可以用于反射或程序化地检查模型定义的字段。
config	一个内嵌的类，用于配置模型的行为，如是否允许任意字段、JSON序列化时的行为等。通过在模型定义中创建一个Config子类来自定义这些设置。

In [ ]:
from datetime import date
from typing import Optional, List
from pydantic import BaseModel, Field, ValidationError, validator
from enum import Enum
import json

class Gender(Enum):
    MALE = '男'
    FEMALE = '女'

class StudentModel(BaseModel):
    sid: int = Field(..., gt=1000, lt=9999, description="学生学号(不可小于1000或大于9999)")
    name: str = Field(..., min_length=2, max_length=10, description="学生姓名(字符不可小于2或大于10)")
    age: int = Field(..., gt=0, lt=100, description="学生年龄(不可小于0或大于100)")
    gender: Gender = Field(default=Gender.MALE, description="学生性别(仅可为男或女)")
    clz: str = "S95001"  # 学生所属班级
    city: Optional[str]  # 学生所属城市
    phone: str = Field(..., min_length=11, max_length=11, description="手机号码(字符仅可为11)")
    birthday: date  # 学生生日
    hobby: List[str] = Field(default=None, min_items=1, max_items=3,
                             description="学生爱好（可以为None或不可小于1或大于3个爱好）")

    @validator('clz')
    def check_clz_startswith(cls, v):
        if not v.startswith("S950"):
            raise ValueError('班级编号必须以S950开头')
        return v

    @validator("hobby")
    def check_hobby_items(cls, hobby, values):
        if values.get("gender") == Gender.MALE and hobby[0] != "篮球":
            raise ValueError("本班级男同学的第一爱好必须是篮球哦")
        return hobby


# 创建一条符合StudentModel模型的学生数据
try:
    student1 = StudentModel(
        sid=1001,
        name="张三",
        age=18,
        city="上海",
        phone="13500001234",
        birthday="1999-11-20",
    )
    print(type(student1), student1)
except ValidationError as e:
    print("Validation Error:", e)

# clz字段，不以S950开头
try:
    student2 = StudentModel(
        sid=1001,
        name="张三",
        age=18,
        clz="10086",
        city="上海",
        phone="13500001234",
        birthday="1999-11-20",
    )
except ValidationError as e:
    error_msg = json.loads(e.json())
    print("发送错误的字段：", error_msg[0]["loc"])  # 发送错误的字段

# gender为男的同学，hobby中不传递篮球或篮球不在第一元素
try:
    student3 = StudentModel(
        sid=1001,
        name="张三",
        age=18,
        gender=Gender.MALE,
        city="上海",
        phone="13500001234",
        birthday="1999-11-20",
        hobby=["电竞", "直播", "户外"]
    )
except ValidationError as e:
    error_msg = json.loads(e.json())
    print("发送错误的字段：", error_msg[0]["loc"])  # 发送错误的字段


### 模型嵌套

In [ ]:
from typing import List
from pydantic import BaseModel


class Student(BaseModel):
    num: str
    name: str


class Teacher(BaseModel):
    num: str
    name: str
    domain: str


class Clazz(BaseModel):
    num: str
    maximum: int


class School(BaseModel):
    clazz: List[Clazz]
    student: List[Student]
    teacher: List[Teacher]


# 创建学生实例
student1 = Student(num="S1001", name="张三")
student2 = Student(num="S1002", name="李四")
student3 = Student(num="S1003", name="王五")

# 创建老师实例，确保每位老师的编号唯一
teacher1 = Teacher(num="T1001", name="赵德柱", domain="数学")
teacher2 = Teacher(num="T1002", name="王铁锤", domain="语文")  # 修改编号
teacher3 = Teacher(num="T1003", name="吴鱼子", domain="英语")  # 修改编号

# 创建班级实例，确保班级编号唯一
clazz1 = Clazz(num="C1001", maximum=30)
clazz2 = Clazz(num="C1002", maximum=40)  # 修改编号

# 创建学校实例
school = School(
    clazz=[clazz1, clazz2],
    student=[student1, student2, student3],
    teacher=[teacher1, teacher2, teacher3]
)

# 输出学校实例的类型和内容
print(type(school), school)

# 输出学校实例的字典表示
print(school.dict())

# 输出学校实例的 JSON 表示
print(school.json())


### json数据导入


In [ ]:
from pathlib import Path
from datetime import date
from typing import Optional, List
from pydantic import BaseModel, Field, ValidationError, validator
from enum import Enum


class Gender(Enum):
    MALE = '男'
    FEMALE = '女'


class StudentModel(BaseModel):
    sid: int = Field(..., gt=1000, lt=9999, description="学生学号(不可小于1000或大于9999)")
    name: str = Field(..., min_length=2, max_length=10, description="学生姓名(字符不可小于2或大于10)")
    age: int = Field(..., gt=0, lt=100, description="学生年龄(不可小于0或大于100)")
    gender: Gender = Field(default="男", description="学生性别(仅可为男或女)")
    clz: str = "S95001"  # 学生所属班级
    city: Optional[str]  # 学生所属城市
    phone: str = Field(None, min_length=11, max_length=11, description="手机号码(字符仅可为11)")
    birthday: date  # 学生生日
    hobby: List[str] = Field(default=None, min_items=1, max_items=3,
                              description="学生爱好（可以为None或不可小于1或大于3个爱好）")

    @validator('clz')
    def check_clz_startswith(cls, v):
        if not v.startswith("S950"):
            raise ValueError('班级编号必须以S950开头')
        return v

    @validator("hobby")
    def check_hobby_items(cls, hobby, values):
        if values["gender"] == Gender.MALE and hobby[0] != "篮球":
            raise ValueError("本班级男同学的第一爱好必须是篮球哦")
        return hobby


# 创建一条符合StudentModel模型的学生数据
student1 = StudentModel(
    sid=1001,
    name="张三",
    age=18,
    city="上海",
    phone="13500001234",
    birthday="1999-11-20",
)
print(type(student1), student1)

# 从指定路径下的json文件中读取内容，通过BaseModel模型的parse_file函数进行解析
path = Path("./example.json")
# example.json文件数据如下：
# {
#   "sid": 1101,
#   "name": "弗兰加圣香克斯",
#   "age": 35,
#   "gender": "男",
#   "clz": "S95001",
#   "city": "北海",
#   "phone": "13500001234",
#   "birthday": "1985-11-20",
#   "hobby": [
#     "篮球",
#     "美食",
#     "户外"
#   ]
# }

file_data = StudentModel.parse_file(path)
print(file_data.name)      # 弗兰加圣香克斯
print(file_data.dict())    # 输出字典格式的数据
print(file_data.json())    # 输出 JSON 格式的数据


### orm数据导入

In [ ]:
from typing import List
from sqlalchemy import Column, Integer, String
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext.declarative import declarative_base
from pydantic import BaseModel, constr

Base = declarative_base()

# 创建一张表（ORM模型）
class CompanyOrm(Base):
    __tablename__ = 'companies'
    id = Column(Integer, primary_key=True, nullable=False)
    public_key = Column(String(20), index=True, nullable=False, unique=True)
    name = Column(String(63), unique=True)
    domains = Column(ARRAY(String(255)))

# 创建一个pydantic的BaseModel模型
class CompanyModel(BaseModel):
    id: int
    public_key: constr(max_length=20)
    name: constr(max_length=63)
    domains: List[constr(max_length=255)]

    class Config:
        orm_mode = True
        from_attributes = True

# 实例化一个数据库表（ORM模型）对象
co_orm = CompanyOrm(
    id=123,
    public_key='foobar',
    name='Testing',
    domains=['example.com', 'foobar.com'],
)
print(type(co_orm), co_orm)

# 使用BaseModel类调用from_orm函数，将CompanyOrm模型解析成CompanyModel模型
company_orm = CompanyModel.from_orm(co_orm)
print(type(company_orm), company_orm)

print(company_orm.dict())  # 输出字典格式的数据
print(company_orm.json())  # 输出 JSON 格式的数据


## sqlalchemy